In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import re
import json
import torch
import pickle
import tensorflow as tf

import keras
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
movies = pd.read_csv('archive/movies_metadata.csv')
print("Shape: ", movies.shape)
print("Features: ", movies.columns)

Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/Users/anshverma/opt/anaconda3/envs/python37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies['original_title']

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
45461                        رگ خواب
45462            Siglo ng Pagluluwal
45463                       Betrayal
45464            Satana likuyushchiy
45465                       Queerama
Name: original_title, Length: 45466, dtype: object

In [4]:
print("Shape: ", movies.shape)
print("Features: ", movies.columns)

Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [5]:
movies['overview'][1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

In [6]:
movies.head()['imdb_id']

0    tt0114709
1    tt0113497
2    tt0113228
3    tt0114885
4    tt0113041
Name: imdb_id, dtype: object

In [7]:
movies.head()['genres']

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [8]:
movies.head()['poster_path']

0    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1    /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2    /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3    /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4    /e64sOI48hQXyru7naBFyssKFxVd.jpg
Name: poster_path, dtype: object

In [9]:
movies.head()['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [10]:
movies.head()['genres'][1]

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [11]:
movies.columns.values

array(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'], dtype=object)

In [ ]:
movies['popularity']

In [12]:
movies.dropna(subset = ['poster_path', 'genres'], inplace=True)

In [13]:
movies.dropna(subset = ["overview"], inplace=True)
print(movies.shape)
print(movies)

movies.fillna(0, inplace=True)

movies = movies[0: 4961]

(44166, 24)
       adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                                  genres  \
0  

In [14]:
movies['poster_path'].isnull().sum()

0

In [23]:
# @Source: https://johannesbader.ch/blog/tutorial-download-posters-with-the-movie-database-api-in-python/"
import os
import re
import requests

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}' 
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()
    
def _download_images(titles, urls, path):
    """download all images in list 'urls' to 'path' """
    filePathList = []
    print(len(urls), len(titles))
    zipped = list(zip(titles, urls))
    i = 0
    for title, url in zipped: 
        r = requests.get(url)
        filetype = r.headers['content-type'].split('/')[-1]
        filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'
        print(i)
        filepath = os.path.join(path, filename)
        filePathList.append(filepath)
        #commented this out so that won't redownload data every single time
        #if r.status_code == 200:
        #    with open(filepath,'wb') as w:
        #        w.write(r.content)
        i+=1
    
    return path, filePathList 

def get_poster_url(imdbid):
    """ 
    given IMDB id, returns the corresponding url for
    poster jpg
    
    Args: 
        imdbid (str): IMDB id
    Return: 
        url (str): image url of poster image from 'themoviedb.org'
                of the maximum available size. 
    """
    config = _get_json(CONFIG_PATTERN.format(key=KEY))
    base_url = config['images']['base_url']
    sizes = config['images']['poster_sizes']   # sorted in ascending order
    
    def size_str_to_int(x):
        return float("inf") if x == 'original' else int(x[1:])
    
    # retrieve image of max size
    max_size = max(sizes, key=size_str_to_int)
    
    try: 
        posters = _get_json(IMG_PATTERN.format(key=KEY,imdbid=imdbid))['posters']
    except: 
        print(imdbid)
    
    if len(posters) == 0:
        return None
    
    rel_path = posters[0]['file_path']
    url = "{0}{1}{2}".format(base_url, max_size, rel_path)
    
    return url

In [ ]:
# RUN THIS FUNCTION TO DOWNLOAD JPG IMAGES
def tmdb_posters():
    """ download posters with imdb_id from dataset """
    
    if not os.path.exists('images_movies_genre'):
        os.makedirs('images_movies_genre')
    filePathList = []
    titles = []
    urls = []
    checkIter = []
    n = 0
    for index, row in movies.iterrows():
        title = str(row['title'])
        url = get_poster_url(row['imdb_id'])
        
        # skip iteration if poster cannot be located
        if not url:
            checkIter.append(0)
            continue
            
        checkIter.append(1)
        titles.append(title)
        urls.append(url)
        n += 1
        #print(n)
        if n == 1000:
            break
            
        # SET LIMIT FOR NUMBER OF IMAGES TO DOWNLOAD

    path, filePathList = _download_images(titles, urls, 'images_movies_genre')
    retList = []
    counter = 0
    for i in checkIter: 
        if i == 0: 
            retList.append('NaN')
        elif i == 1: 
            retList.append(filePathList[counter])
            counter+=1
    return path, retList

path, filePathList = tmdb_posters() 

In [ ]:
filename = "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/ImageData"
filenameTwo = "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/Genres"
fileNameThree =  "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/xTrain"
fileNameFour =  "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/xTest"
fileNameFive =  "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/yTrain"
fileNameSix =  "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/yTest"

def saveImages(imageList, genres, pathOne, pathTwo): 
    filehandler = open(pathOne, 'wb')
    np.save(filehandler, imageList)
    filehandlerTwo = open(pathTwo, 'wb')
    np.save(filehandlerTwo, genres)
    #pickle.dump(imageList, filehandler)
        
def loadImages(): 
    filehandler = open(filename, 'rb') 
    files = np.load(filehandler, allow_pickle=True) #pickle.load(filehandler)
    return files

def loadGenres(): 
    filehandlerTwo = open(filenameTwo, 'rb')
    genres = np.load(filehandlerTwo, allow_pickle=True)
    return genres

def load(filePath): 
    loader = csv

def tempSave(genres): 
    genres.to_csv(path_or_buf = filenameTwo)
    
def saveTesting(csv, path): 
    csv.to_csv(path_or_buf = path)
    
def fromCsv(path): 
    return pd.read_csv(path)

In [3]:
# Converting images to array
X = []

for path in tqdm(filePathList):
    try: 
        img = image.load_img(path, target_size=(224, 224, 3))
        img = image.img_to_array(img)
        img = img/255.0
        X.append(img)
    except: 
        X.append('NaN')
        continue
    
X = np.array(X)

NameError: name 'tqdm' is not defined

In [19]:
len(X)

NameError: name 'X' is not defined

In [20]:
movies.insert(len(movies.columns), "imageArrays", X, True)

NameError: name 'X' is not defined

In [14]:
#Padding
#reference: https://ai-pool.com/d/padding-images-with-numpy
def pad(img, h, w):
    top_pad = np.floor((h - img.shape[0]) / 2).astype(np.uint16)
    bottom_pad = np.ceil((h - img.shape[0]) / 2).astype(np.uint16)
    right_pad = np.ceil((w - img.shape[1]) / 2).astype(np.uint16)
    left_pad = np.floor((w - img.shape[1]) / 2).astype(np.uint16)
    return np.copy(np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode='constant', constant_values=0))

In [15]:
#checking if padding works
img = image.load_img('images_movies_genre/toy-story.jpg', target_size=(224, 224, 3))
new_img_array=pad(X[0], 512, 512)
new_img=image.array_to_img(new_img_array)
new_img.show()

NameError: name 'X' is not defined

In [16]:
import ast
#reference-https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
def genre_list(str):
    arr=ast.literal_eval(str)
    genreList=[dict['name'] for dict in arr]
    return genreList

In [17]:
genre_list_collection=[]
for i in movies['genres']:
    genre_list_collection+=[genre_list(i)]
genre_list_collection[0]

['Animation', 'Comedy', 'Family']

In [18]:
#adding a column called genre_list which contains all the genres of the movies in a list
movies.insert(5, "genre_list", genre_list_collection, True)
print("Features: ", movies.columns)
movies['genre_list']

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'genre_list', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


0                [Animation, Comedy, Family]
1               [Adventure, Fantasy, Family]
2                          [Romance, Comedy]
3                   [Comedy, Drama, Romance]
4                                   [Comedy]
                        ...                 
5034                                [Horror]
5035                [Romance, Drama, Comedy]
5036    [Comedy, Thriller, Mystery, Romance]
5037                [Comedy, Music, Romance]
5038                                 [Drama]
Name: genre_list, Length: 5000, dtype: object

In [19]:
all_genres = sum(genre_list_collection,[])
len(set(all_genres))

20

In [20]:
#One hot encoding the genres
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(movies['genre_list'].values)
genreLabelsList=mlb.transform(movies['genre_list'])
print(genreLabelsList.shape)
#TODO Add the one hot encoded genres to the movies dataframe
genre_labels=pd.DataFrame(genreLabelsList, columns=mlb.classes_)
print(genre_labels)

(5000, 20)
      Action  Adventure  Animation  Comedy  Crime  Documentary  Drama  Family  \
0          0          0          1       1      0            0      0       1   
1          0          1          0       0      0            0      0       1   
2          0          0          0       1      0            0      0       0   
3          0          0          0       1      0            0      1       0   
4          0          0          0       1      0            0      0       0   
...      ...        ...        ...     ...    ...          ...    ...     ...   
4995       0          0          0       0      0            0      0       0   
4996       0          0          0       1      0            0      1       0   
4997       0          0          0       1      0            0      0       0   
4998       0          0          0       1      0            0      0       0   
4999       0          0          0       0      0            0      1       0   

      Fantasy  F

In [21]:
genre_labels.to_csv('GenreLabels.csv', index=False)

In [35]:
mlb.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [36]:
overviews=pd.DataFrame(movies["overview"])
movies_with_overviews=pd.concat([overviews, genre_labels], axis=1)
movies_with_overviews.head()
#movies_with_overveiws=genre_labels
#movies_with_overviews["overview"]=movies["overview"]

,overview,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,"Led by Woody, Andy's toys live happily in his ...",0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,When siblings Judy and Peter discover an encha...,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,A family wedding reignites the ancient feud be...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,"Cheated on, mistreated and stepped on, the wom...",0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Just when George Banks has recovered from his ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
overviews=pd.DataFrame(movies["overview"])
movies_with_overviews=pd.concat([overviews, genre_labels], axis=1)
movies_with_overviews.head()
movies_with_overviews.dropna(subset = ["Action"], inplace=True)
#movies.dropna(subset = ["overview"], inplace=True)

In [38]:
movies_with_overviews

,overview,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,"Led by Woody, Andy's toys live happily in his ...",0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,When siblings Judy and Peter discover an encha...,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,A family wedding reignites the ancient feud be...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,"Cheated on, mistreated and stepped on, the wom...",0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Just when George Banks has recovered from his ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,The calving of an Arctic iceberg releases a gi...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4996,A grieving doctor is being contacted by his la...,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4997,Lestat de Lioncourt is awakened from his slumb...,0,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0,0
4998,Vietnam veteran Leon Barlow is struggling as a...,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [41]:
filename = "/Users/anshverma/Documents/GitHub/MovieGenrePrediction/ImageData"
import numpy as np
def loadImages(): 
    filehandler = open(filename, 'rb') 
    files = np.load(filehandler, allow_pickle=True) #pickle.load(filehandler)
    return files

In [42]:
newX = loadImages()

In [43]:
print("Shape: ", movies_with_overviews.shape)
print("Features: ", movies_with_overviews.columns)
movies_with_overviews.to_csv('MoviesWithOverviews.csv', index=False)

Shape:  (5000, 21)
Features:  Index(['overview', 'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie',
       'Thriller', 'War', 'Western'],
      dtype='object')


In [44]:
movies.shape

(4961, 25)

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anshverma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
# Text cleaning
#reference: https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text): 
    #removing everything except letters and converting to lower case
    text = re.sub("[^a-zA-Z]"," ",str(text))
    text = ' '.join(text.split())
    text = text.lower() 
    return text

# Removing stopwords such as the, and, to, or etc
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [47]:
plot_summaries=[]
for text in movies['overview']:
    cleanText=clean_text(text)
    textWithoutStopwords=remove_stopwords(cleanText)
    plot_summaries+=[textWithoutStopwords]
movies.insert(11, "plot_summary", plot_summaries, True)
print("Features: ", movies.columns)

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'genre_list', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'plot_summary', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count'],
      dtype='object')


In [34]:
movies["plot_summary"]

KeyError: 'plot_summary'

In [32]:
X_train, X_test, y_train, y_test = train_test_split(movies, genre_labels, test_size=0.20)

In [ ]:
#saveImages(newX, movies['genre_list']) 
saveImages(X_train, X_test, fileNameThree, fileNameFour)
saveImages(y_train, y_test, fileNameFive, fileNameSix)

In [ ]:
tempSave(genre_labels)

In [ ]:
images = loadImages()

In [ ]:
genres = loadGenres()

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tensor_x = torch.Tensor(genre_labels.iloc[:len(newX)].values) # transform to torch tensor
tensor_y = torch.Tensor(newX)
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset

In [ ]:
import torch
import torch.nn as nn
#from torchsummary import summary

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=32):
        super(ResNet, self).__init__()
        self.block = block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #ResNet layers
        self.layer1 = self.new_layer(num_blocks=layers[0], in_channels=64, out_channels=64, stride=1)
        self.layer2 = self.new_layer(num_blocks=layers[1], in_channels=64, out_channels=128, stride=2)
        self.layer3 = self.new_layer(num_blocks=layers[2], in_channels=128, out_channels=256, stride=2)
        self.layer4 = self.new_layer(num_blocks=layers[3], in_channels=256, out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        print(x)
        print(type(x))
        #print("input: ", x.shape)
        x = self.conv1(x)
        #print("conv1: ", x.shape)
        x = self.bn1(x)
        #print("bn1: ", x.shape)
        x = self.relu(x)
        #print("relu: ", x.shape)
        x = self.maxpool(x)
        #print("maxpool: ", x.shape)

        x = self.layer1(x)
        #print("layer 1: ", x.shape)
        x = self.layer2(x)
        #print("layer 2: ", x.shape)
        x = self.layer3(x)
        #print("layer 3: ", x.shape)
        x = self.layer4(x)
        #print("layer 4: ", x.shape)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x
    
    def new_layer(self, num_blocks, in_channels, out_channels, stride):
        layers = []
        layers.append(self.block(in_channels, out_channels))
        intermediate_channels = out_channels
        for i in range(num_blocks):
            layers.append(self.block(intermediate_channels, out_channels))
           
        return nn.Sequential(*layers)

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(Block, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.conv4 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn4 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.identity = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0)

    def forward(self, x):
        identity = x.clone()
        if self.in_channels != self.out_channels:
            identity = self.identity(identity)
        #print("id1", identity.shape)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        #Skip connection 1
        #print("id2", identity.shape, x.shape)
        x += identity
        identity = x.clone()
        #print("id3", identity.shape)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)

        #Skip connection 2
        x += identity

        x = self.relu(x)

        return x

def ResNet34():
    return ResNet(Block, [3, 4, 6, 3])

model = ResNet34()
#summary(model, input_size=(3, 224, 224), batch_size=1)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets
#from data_pytorch import Data
#from rotnet import RotNet
import time
import shutil
import yaml

#parser = argparse.ArgumentParser(description='Configuration details for training/testing rotation net')
#parser.add_argument('--config', type=str, required=True)
#parser.add_argument('--train', action='store_true')
#parser.add_argument('--data_dir', type=str, required=True)
#parser.add_argument('--image', type=str)
#parser.add_argument('--model_number', type=str, required=True)

#args = parser.parse_args()

#config = yaml.load(open(args.config, 'r'), Loader=yaml.FullLoader)

"""
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    
    #for batch in train_dataloader:
    for i, data in enumerate(train_dataloader, 0):
        #print(i)
        #print(data)
        #X_batch, y_batch = batch[0], batch[1] #.view(-1, 784)
        #print(y_batch)
        #print(X_batch, y_batch)
        #print(X_batch, y_batch)
        # THESE 5 LINES ARE IMPORTANT, UNDERSTAND WHAT IS HAPPENING HERE
        optimizer.zero_grad()
        predicted_batch = model(data) 
        #print(predicted_batch)
        loss = criterion(predicted_batch, y_batch)
        #print(loss)
        loss.backward()
        optimizer.step()
        total_loss += loss
        
        
       
    print("Epoch {0}: {1}".format(epoch, total_loss))
    #if epoch%5 == 0 and epoch != 0:
    #    test_batch = next(iter(test_dataloader))
    #    X_test, y_test = test_batch[0].view(-1, 784), test_batch[1]
    #    predicted = model(X_test)
    #    test_acc = torch.sum(y_test == torch.argmax(predicted, dim=1), dtype=torch.double) / len(y_test)
    #    print("\tTest Accuracy {0}".format(test_acc))
"""

def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for i, (test, train) in enumerate(train_loader):
        optimizer.zero_grad()
        predicted_batch = model(train)
        loss = criterion(predicted_batch, test)
        loss.backward()
        optimizer.step()
        total_loss += loss
        print(total_loss)
    """ 
    for i, (input, target) in enumerate(train_loader):
        optimizer.zero_grad()
        predicted_batch = model(input)
        loss = criterion(predicted_batch, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss
    """

def validate(val_loader, model, criterion):
    model.eval()
    for i, (input, target) in enumerate(val_loader):
        optimizer.zero_grad()
        outputs = model(input)
        #TODO: implement the validation. Remember this is validation and not training
        #so some things will be different.

def save_checkpoint(state, best_one, filename='rotationnetcheckpoint.pth.tar', filename2='rotationnetmodelbest.pth.tar'):
    torch.save(state, filename)
    #best_one stores whether your current checkpoint is better than the previous checkpoint
    if best_one:
        shutil.copyfile(filename, filename2)

def main():
    n_epochs = 10 #config["num_epochs"]
    model = ResNet34()
    criterion = torch.nn.MSELoss() #what is your loss function
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum = 0.9) #temp
    train_dataset = loadImages()
    train_dataloader = DataLoader(my_dataset, batch_size=256, shuffle=True)
    #val_loader = #how will you get your dataset
    #val_loader = #how will you use pytorch's function to build a dataloader
    for epoch in range(n_epochs):
            train(train_dataloader, model, criterion, optimizer, epoch)
            
         #TODO: make your loop which trains and validates. Use the train() func

         #TODO: Save your checkpoint

if __name__ == "__main__":
    main()

In [ ]:
for i in range(len(movies['popularity'])): 
    movies['popularity'][i] = movies['popularity'][i]/10

In [ ]:
movies['popularity']

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        #self.conv1 = nn.Conv2d(1, 6, 5)
        #self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        #self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc1 = nn.Linear(20, 15)
        self.fc2 = nn.Linear(15, 10)
        #self.fc2 = nn.Linear(16, )
        #self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #x = x.view(-1, self.num_flat_features(x))
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

    #def num_flat_features(self, x):
    #    size = x.size()[1:]  # all dimensions except the batch dimension
    #    num_features = 1
    #    for s in size:
    #        num_features *= s
    #    return num_features

net = Net()
print(net) 

In [ ]:
train_dataloader = DataLoader(my_dataset, batch_size=256, shuffle=True)
model1 = ResNet34()
nlpModel = tf.keras.models.load_model('savedNlpModel')
finalTrainingData = []
for i, (test, train) in enumerate(train_dataloader):
    genre1 = model1(train)
    genre2 = nlpModel(train)
    finalTrainingData.append(genre1 + genre2)

finalTrainingData = np.array(finalTrainingData)

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tensor_x = torch.Tensor(finalTrainingData) # transform to torch tensor
tensor_y = torch.Tensor(movies['popularity'])
my_dataset = TensorDataset(tensor_x,tensor_y) # create your dataset

In [ ]:
def main():
    n_epochs = 10 #config["num_epochs"]
    model = Net() #
    criterion = torch.nn.MSELoss() #what is your loss function
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001) #temp
    train_dataset = loadImages()
    train_dataloader = DataLoader(my_dataset, batch_size=256, shuffle=True)
    #val_loader = #how will you get your dataset
    #val_loader = #how will you use pytorch's function to build a dataloader
    for epoch in range(n_epochs):
            train(train_dataloader, model, criterion, optimizer, epoch)
            
         #TODO: make your loop which trains and validates. Use the train() func

         #TODO: Save your checkpoint

main()

In [ ]:
for value in range(len(epoch)):
    net(model)

In [ ]:
def resNetOutput():
    for image in images: 
        genre = model(image)